# Table join processing task

In this notebook, we will demonstrate joining a table of internet users per NUTS area with UK local authority polygons.

Datasets used:

- Office of National Statistics [Internet users by NUTS code](https://www.ons.gov.uk/file?uri=/businessindustryandtrade/itandinternetindustry/datasets/internetusers/current/internetusers2020.xlsx)
- [NUTS Code conversions](https://geoportal.statistics.gov.uk/datasets/ons::local-authority-district-december-2016-to-nuts3-to-nuts2-to-nuts1-january-2018-lookup-in-united-kingdom-1/explore)
- [UK authority polygon shapefiles](https://geoportal.statistics.gov.uk/datasets/e204895bba5646a486da29b5ed382db1_0/explore)

## 0. Install Python packages

In [ ]:
!pip install geopandas folium

## 1. Load data

First connect Databricks to your datalake.

In [ ]:
datalake = "/dbfs/mnt/copgeospatial"

In [ ]:
import pandas as pd 
import geopandas as gpd

Internet users by NUTS code

In [ ]:
df_internet = pd.read_csv(f"{datalake}/internetusers2020_updt.csv")

df_internet.columns = [
    "NUTS_Code", "Region_Name", 
    "2014", "2015", "2016", "2017", "2018", "2019", "2020"
]
df_internet.head()

NUTS to local administrative unit conversion (columns `LAU118CD`, `NUTS318CD`):

In [ ]:
df_nuts = pd.read_csv(f"{datalake}/LAU2_to_LAU1_to_NUTS3_to_NUTS2_to_NUTS1_(December_2018)_Lookup_in_United_Kingdom.csv") \

df_nuts = df_nuts[["LAU118CD", "NUTS318CD"]].drop_duplicates()
df_nuts.sort_values(by="NUTS318CD").head()

UK region shapefiles stored in a `geodataframe` (with `geometry` column):

In [ ]:
gdf = gpd.read_file(f"{datalake}/Counties_and_Unitary_Authorities_(December_2022)_UK_BFC/Counties_and_Unitary_Authorities_(December_2022)_UK_BFC.shp")
gdf.head()

## 2. Process data

Merge 3 datasets: local authority vector polygons, NUTS codes and internet users per NUTS code:

In [ ]:
df_merged = df_internet \
    .merge(df_nuts, left_on="NUTS_Code", right_on="NUTS318CD", how="left") \
    .merge(gdf, left_on="LAU118CD", right_on="CTYUA22CD", how="right")

df_merged = df_merged[[
    'Region_Name', '2014', '2015', '2016', '2017', '2018', '2019', '2020',
    'LAU118CD', 'Shape__Are', 'Shape__Len', 'GlobalID', 'geometry'
]].astype(str)

Convert the merged `dataframe` to a `geodataframe`:

In [ ]:
from shapely.wkt import loads

In [ ]:
df_merged['GlobalID'] = df_merged['GlobalID'].apply(lambda x: str(x))
df_merged['2020']     = df_merged['2020'    ].apply(lambda x: float(x))
df_merged['geometry'] = df_merged['geometry'].apply(loads)

In [ ]:
gdf_merged = gpd.GeoDataFrame(df_merged, geometry='geometry').set_crs('epsg:27700')

Simplify geometries for faster plotting and convert CRS for plotting on top of world maps:

In [ ]:
gdf_merged["geometry"] = gdf_merged["geometry"].simplify(tolerance=100, preserve_topology=False)

In [ ]:
gdf_merged = gdf_merged.to_crs(epsg=4326)

## 3. Visualise data on map

In [1]:
import folium

In [ ]:
m = folium.Map(location=[54.44, -3.5], width=500, height=750, zoom_start=12, tiles="CartoDB positron")

layer = folium.Choropleth(
    geo_data = gpd.GeoSeries(gdf_merged.set_index("GlobalID")['geometry']).to_json(),
    name="Choropleth",
    data=gdf_merged,
    key_on='feature.id',
    columns=['GlobalID', '2020'],
    bins=[0, 50, 150, 250, 300, 550, 1000],
    fill_color='YlGnBu',
    fill_opacity=0.5,
    line_opacity=1.0,
    legend_name='Internet Users in millions').add_to(m)

folium.LayerControl().add_to(m)
m.fit_bounds(layer.get_bounds()) 

In [ ]:
m.save('/dbfs/mnt/copgeospatial/01_output.html')

### Display map

In [1]:
from IPython.display import IFrame
IFrame("01_output.html", width="100%", height="700")